In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.abspath(".."))

In [7]:
from minrl.algorithms import (
    rollout
)
from minrl.tasks.connections import ConnectionsDataset


dataset = ConnectionsDataset(split="eval", host="notebook")

2025-10-12 20:49:50.886 | INFO     | minrl.tasks.connections:__init__:123 - Loading generated prompts from ../data/connections_generated.jsonl
2025-10-12 20:49:50.905 | INFO     | minrl.tasks.connections:__init__:129 - Generating 1000 samples
2025-10-12 20:49:51.371 | INFO     | minrl.tasks.connections:__init__:144 - Loading canonical prompts from ../data/connections_canonical.json


In [ ]:
batch = dataset[:4]

conversations = [
    dataset.initial_conversation(sample, i)
    for i, sample in enumerate(batch)
]

episodes = rollout(
    temperature,
    max_new_tokens,
    tokenizer,
    group_size,
    1,
    conversations,
    batch,
    reward_function=self.reward_function,
    vllm_model=self.vllm_model,
    prev_reward_std=prev_reward_std,
)

[{'role': 'system', 'content': '\nFind four groups of four words that are related to each other.\nYou may only use each word in one group.\nAlways return four groups of four words.\nRespond in the following format:\n<answer>\n<group>\n...\n</group>\n<group>\n...\n</group>\n</answer>\n\n'}, {'role': 'user', 'content': 'candle, crayon, honeycomb, seal, kettles, mittens, raindrops, whiskers, canine, fang, molar, tusk, defense, excuse, out, reason'}, {'role': 'assistant', 'content': '\n<answer>\n<group>candle, crayon, honeycomb, seal</group>\n<group>kettles, mittens, raindrops, whiskers</group>\n<group>canine, fang, molar, tusk</group>\n<group>defense, excuse, out, reason</group>\n</answer>\n'}, {'role': 'user', 'content': 'dance, meal, one, root, bite, edge, punch, spice, mineral, spring, still, tap, hi, la, ma, ok'}]
